# Extraction + Learning : Hardware Spec Sheets

This notebook demonstrates the full entity extraction process on transistor data sheets, extracting min storage temperatures.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from snorkel import SnorkelSession
# session = SnorkelSession()

### Extraction

In [1]:
from snorkel.parser import HTMLParser
html_parser = HTMLParser(path='data/hardware/hardware_docs/')

In [2]:
from snorkel.parser import TableParser
table_parser = TableParser()

In [3]:
try:
    import cPickle
    with open("hardware_corpus.pkl","r") as pkl:
        %time corpus = cPickle.load(pkl)
    print "Corpus has been loaded."
except:
    from snorkel.parser import CorpusParser
    cp = CorpusParser(html_parser, table_parser, max_docs=200)
    %time corpus = cp.parse_corpus(name='Hardware Corpus')
    print "Corpus has been parsed."

CPU times: user 53.1 s, sys: 2.35 s, total: 55.5 s
Wall time: 55.8 s
Corpus has been loaded.


In [ ]:
# import cPickle
# with open("hardware_corpus.pkl",'wb') as pkl:
#     cPickle.dump(corpus, pkl)

In [ ]:
# session.add(corpus)
# session.commit()

In [ ]:
# from snorkel.models import Corpus
# corpus = session.query(Corpus).filter(Corpus.name == 'Hardware Corpus').one()
# corpus

In [ ]:
from snorkel.candidates import TableNgrams
from snorkel.matchers import NumberMatcher, RangeMatcher

# Define a candidate space
ngrams = TableNgrams(n_max=3)

# Define a matcher
number_matcher = RangeMatcher(low=-70,high=-50)

In [ ]:
# from snorkel.candidates import Candidates
# %time candidates = Candidates(ngrams, number_matcher, corpus.get_contexts())
# for c in candidates.get_candidates()[:5]: print c
    
from snorkel.candidates import CandidateExtractor
ce = CandidateExtractor(ngrams, number_matcher)
%time candidates = ce.extract(corpus.get_tables(), name='all')
for cand in candidates[:10]: 
    print cand
print "%s candidates extracted" % len(candidates)

In [ ]:
# session.add(candidates)
# session.commit()

In [ ]:
# from snorkel.models import CandidateSet
# candidates = session.query(CandidateSet).filter(CandidateSet.name == 'all').one()
# print len(candidates)

### Learning

First, generate gold data.

In [ ]:
from utils import collect_hardware_temperature_annotations
gt_dict = collect_hardware_temperature_annotations('data/hardware/gold_all.csv', 'stg_temp_min', candidates)
print "%s out of %s candidates have gold labels" % (len(gt_dict),len(candidates))
print "%s out of %s labeled candidates have positive label" \
        % (gt_dict.values().count(1), len(gt_dict))

Then confirm that features work:

In [ ]:
from snorkel.features import TableNgramFeaturizer
featurizer = TableNgramFeaturizer()
featurizer.fit_transform(candidates)
for f in featurizer.get_features_by_candidate(candidates[0])[:10]: print f

We now build a legacy DDLiteModel with which we will do learning.

In [ ]:
from snorkel.snorkel import DDLiteModel
DDL = DDLiteModel(candidates, featurizer.fit_transform(candidates), gt_dict)

DDL.set_holdout(idxs=range(int(len(gt_dict)*.7)), validation_frac=0.5)
print "%s training data" % len(DDL.gt.get_labeled_ground_truth('training')[0])
print "%s test data" % len(DDL.gt.get_labeled_ground_truth('test')[0])
print "%s validation data" % len(DDL.gt.get_labeled_ground_truth('validation')[0])

Define labeling functions:

In [ ]:
def LF_to(m):
    return 1 if 'to' in m.post_window('words') else 0
def LF_storage(m):
    return 1 if 'storage' in m.aligned_ngrams('words') else -1
def LF_tstg(m):
    return 1 if 'tstg' in m.aligned_ngrams('words') else -1
def LF_temperature(m):
    return 1 if 'temperature' in m.aligned_ngrams('words') else -1

In [ ]:
LFs = [LF_to, LF_storage, LF_tstg, LF_temperature]
DDL.apply_lfs(LFs, clear=True)

In [ ]:
print DDL.lf_names
DDL.print_lf_stats()

In [ ]:
DDL.top_conflict_lfs(n=3)

In [ ]:
DDL.lowest_coverage_lfs(n=3)

Now learn, baby, learn!

In [ ]:
import matplotlib
import numpy as np
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12,4)
mu_seq = np.ravel([1e-9, 1e-5, 1e-3, 1e-1])
lf_opts = {'sample': False, 'verbose': True}
model_opts = {'sample': False, 'n_iter': 3000, 'alpha': 0.5, 'mu': mu_seq, 'bias': False, 'verbose': True}
%time DDL.train_model(method='lr', lf_opts=lf_opts, model_opts=model_opts)

In [ ]:
DDL.show_log()
idxs, gt = DDL.get_labeled_ground_truth(subset=DDL.holdout())
acc_feats = np.mean(DDL.get_predicted(subset=DDL.holdout()) == gt)
print "Full model accuracy: {:.3f}".format(acc_feats)

Tune in next time for relation extraction!